# Incremental Dynamic Analysis (IDA) and Fragility Curve for a Nonlinear SDOF System

This notebook performs an Incremental Dynamic Analysis (IDA) on a single-degree-of-freedom (SDOF) system with nonlinear properties using FEMA P695 far-field ground motions from NGA-West2. For each ground motion record, the model is scaled from 0.05g to 2.0g (in increments of 0.05g), the maximum acceleration response is recorded, and collapse is determined based on a predefined collapse deformation. Using these results, the code constructs both IDA curves and a fragility curve (probability of collapse vs intensity measure). Rayleigh damping is applied to achieve a 5% damping ratio.

The code is modular and organized into functions for:

- Reading ground motion records
- Building the SDOF model
- Running the dynamic analysis for a given intensity
- Processing results to build IDA and fragility curves

Author: **Mohammad Talebi-Kalaleh** (<talebika@ualberta.ca>)

Feel free to contact me for inquiries.

See the README for further details on capabilities and citation information.

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import openseespy.opensees as ops
import glob

# -----------------------------
# Global SDOF Properties (randomized for this example)
# -----------------------------
np.random.seed(0)  # for reproducibility

# Define fundamental period and compute stiffness
T = np.random.uniform(0.5, 1.5)        # fundamental period in seconds
m = 1.0                                # mass
omega = 2 * math.pi / T                # circular frequency
K = m * (omega ** 2)                   # stiffness

# Define yield strength and post-yield ratio
Fy = np.random.uniform(0.3, 0.6) * 9.81 * m   # yield force (N)
post_yield_ratio = np.random.uniform(0.02, 0.10)  # post-yield stiffness ratio

# Define collapse force (a factor of Fy)
collapse_factor = np.random.uniform(1.2, 2.0)   
F_collapse = collapse_factor * Fy               

print(f"SDOF Properties: T={T:.2f}s, K={K:.1f} N/m, Fy={Fy:.2f} N, postYieldRatio={post_yield_ratio:.3f}, F_collapse={F_collapse:.2f} N")

## Function Definitions

Below are the functions that perform each part of the analysis. They are organized into:

1. **Rayleigh Damping Coefficients**: Computes the Rayleigh damping coefficients to achieve a target damping ratio.
2. **Ground Motion Reader**: Reads a ground motion file in the NGA-West2 (.AT2) format.
3. **Dynamic Analysis**: Builds the SDOF model, applies the ground motion, runs the transient analysis, and returns the peak acceleration and collapse flag.
4. **IDA and Fragility Analysis**: Loops over records and intensities to construct IDA curves and the collapse fragility curve.

In [ ]:
def rayleigh_coefficients(zeta, freq1, freq2):
    """
    Compute Rayleigh damping coefficients (alphaM, betaK) to achieve damping ratio zeta 
    at two circular frequencies: freq1 and freq2.
    """
    w1, w2 = freq1, freq2
    A = np.array([[0.5 * (1.0/w1), 0.5 * w1],
                  [0.5 * (1.0/w2), 0.5 * w2]])
    b = np.array([zeta, zeta])
    alphaM, betaK = np.linalg.solve(A, b)
    return alphaM, betaK

# Compute Rayleigh coefficients for 5% damping at fundamental and a higher mode (5x fundamental frequency)
damping_ratio = 0.05
freq1 = omega
freq2 = 5 * omega
alphaM, betaK = rayleigh_coefficients(damping_ratio, freq1, freq2)

def read_ground_motion(file_path):
    """
    Reads a ground motion file in NGA-West2 .AT2 format.
    Returns the acceleration time series (in g) and time step (dt in seconds).
    """
    with open(file_path, 'r') as f:
        lines = f.readlines()
    dt = None
    for line in lines:
        if 'NPTS' in line and 'DT=' in line:
            parts = line.strip().split(',')
            for part in parts:
                if 'DT=' in part:
                    dt_str = part.split('=')[1].strip().split()[0]
                    dt = float(dt_str)
            break
    header_lines = 0
    for i, line in enumerate(lines):
        if 'NPTS' in line and 'DT=' in line:
            header_lines = i + 1
            break
    accel_data = []
    for line in lines[header_lines:]:
        vals = line.strip().split()
        accel_data.extend([float(val) for val in vals])
    accel_array = np.array(accel_data, dtype=float)
    return accel_array, dt

def run_sdo_analysis(accel_g, dt, scale_factor):
    """
    Build and analyze the SDOF model in OpenSees for a given ground motion and scale factor.
    Returns the peak absolute acceleration (in g) and a flag indicating collapse.
    """
    ops.wipe()
    ops.model('Basic', '-ndm', 1, '-ndf', 1)
    ops.node(1, 0.0); ops.fix(1, 1)
    ops.node(2, 0.0); ops.mass(2, m)
    
    # Define nonlinear material: Steel01 with bilinear behavior
    matTag = 1
    ops.uniaxialMaterial('Steel01', matTag, K, Fy, post_yield_ratio)
    
    # Define collapse by limiting deformation using MinMax material
    yield_disp = Fy / K
    if post_yield_ratio > 1e-6:
        ult_disp = yield_disp + (F_collapse - Fy) / (K * post_yield_ratio)
    else:
        ult_disp = yield_disp if F_collapse <= Fy else 1e6
    ops.uniaxialMaterial('MinMax', 2, matTag, '-max', ult_disp)
    
    ops.element('zeroLength', 1, 1, 2, '-mat', 2, '-dir', 1)
    
    # Apply Rayleigh damping
    ops.rayleigh(alphaM, betaK, 0.0, 0.0)
    
    # Define ground motion: scale the input acceleration time series
    g_accel = 9.81  
    ops.timeSeries('Path', 1, '-dt', dt, '-values', *accel_g, '-factor', scale_factor * g_accel)
    ops.pattern('UniformExcitation', 1, 1, '-accel', 1)
    
    # Set up transient analysis using Newmark integration
    ops.integration('Newmark', 0.5, 0.25)
    ops.system('Diagonal')
    ops.numberer('Plain')
    ops.constraints('Plain')
    ops.algorithm('Newton', '-initial')
    ops.test('NormDispIncr', 1e-7, 10, 3)
    ops.analysis('Transient')
    
    n_steps = len(accel_g)
    ok = ops.analyze(n_steps, dt)
    peak_rel_acc = 0.0
    collapse_occurred = False
    if ok != 0:
        collapse_occurred = True
    
    if ok == 0:
        peak_rel_acc = 0.0
        for step in range(n_steps):
            rel_acc = ops.nodeAccel(2, 1)
            if abs(rel_acc) > abs(peak_rel_acc):
                peak_rel_acc = rel_acc
            ops.analyze(1, dt)
    else:
        ops.wipeAnalysis()
        ops.constraints('Plain')
        ops.numberer('Plain')
        ops.system('Diagonal')
        ops.algorithm('Newton', '-initial')
        ops.test('NormDispIncr', 1e-7, 10)
        ops.analysis('Transient')
        peak_rel_acc = 0.0
        for step in range(n_steps):
            if ops.analyze(1, dt) != 0:
                collapse_occurred = True
                break
            rel_acc = ops.nodeAccel(2, 1)
            if abs(rel_acc) > abs(peak_rel_acc):
                peak_rel_acc = rel_acc
    
    abs_peak_acc = abs(peak_rel_acc) + (scale_factor * max(abs(accel_g)) * 9.81)
    return abs_peak_acc / 9.81, collapse_occurred

def perform_IDA_analysis(record_files, intensity_levels):
    """
    Loops over all provided ground motion files and intensity levels.
    Returns the IDA results and an array of collapse intensities for each record.
    """
    ida_results = []
    collapse_IMs = []
    
    for rec_file in record_files:
        rec_name = rec_file.split('/')[-1]
        accel_g, dt = read_ground_motion(rec_file)
        orig_PGA = max(abs(accel_g))
        record_IMs = []
        record_max_acc = []
        record_collapsed = False
        collapse_im = None
        
        for IM in intensity_levels:
            scale_factor = IM / orig_PGA
            max_acc_response, collapsed = run_sdo_analysis(accel_g, dt, scale_factor)
            record_IMs.append(IM)
            record_max_acc.append(max_acc_response)
            if collapsed and not record_collapsed:
                record_collapsed = True
                collapse_im = IM
                break
        if collapse_im is None:
            collapse_im = 2.1
        collapse_IMs.append(collapse_im)
        ida_results.append({
            'record': rec_name,
            'IM': record_IMs,
            'max_acc': record_max_acc,
            'collapsed': record_collapsed
        })
        print(f"Record {rec_name}: {'Collapsed at {:.2f}g'.format(collapse_im) if record_collapsed else 'No collapse up to 2.0g'}")
    
    return ida_results, np.array(collapse_IMs)

def compute_fragility(collapse_IMs, intensity_levels):
    """
    Computes empirical collapse probabilities and fits a lognormal fragility curve
    based on collapse intensities for each record.
    Returns the intensity points, empirical probabilities, and fitted curve parameters.
    """
    N_records = len(collapse_IMs)
    prob_exceedance = []
    for IM in intensity_levels:
        count_collapsed = np.sum(collapse_IMs <= IM)
        prob_exceedance.append(count_collapsed / N_records)
    
    # Fit lognormal fragility curve (using values capped at 2.0g for records not collapsing)
    caps_for_fit = np.minimum(collapse_IMs, 2.0)
    median_theta = math.exp(np.mean(np.log(caps_for_fit)))
    beta = np.std(np.log(caps_for_fit))
    
    def fragility_cdf(IM):
        return 0.5 * (1 + math.erf((math.log(IM/median_theta)) / (beta * math.sqrt(2))))
    
    IM_plot = np.linspace(0.0, 2.0, 100)
    fragility_curve = [fragility_cdf(x) for x in IM_plot]
    
    return intensity_levels, prob_exceedance, IM_plot, fragility_curve, median_theta, beta


## Main Execution

The main part of the notebook executes the following steps:

1. Read all ground motion records (assumed to be stored in a directory named `Records/` with `.AT2` files).
2. Loop over intensity levels (0.05g to 2.0g) for each record and perform the dynamic analysis using `perform_IDA_analysis`.
3. Compute the fragility curve based on the collapse intensities using `compute_fragility`.
4. Plot the IDA curves and the fragility curve.


In [ ]:
# Main Execution

# Define intensity levels (in g) from 0.05g to 2.0g
intensity_levels = np.arange(0.05, 2.01, 0.05)

# List ground motion files in the 'Records' directory
record_files = glob.glob("Records/*.AT2")
if len(record_files) == 0:
    print("No ground motion files found in the 'Records' directory.")
else:
    # Perform IDA analysis for all records
    ida_results, collapse_IMs = perform_IDA_analysis(record_files, intensity_levels)
    
    # Compute fragility data
    IM_points, prob_exceedance, IM_plot, fragility_curve, median_theta, beta = compute_fragility(collapse_IMs, intensity_levels)
    
    # Plot IDA curves
    plt.figure(figsize=(8,6))
    for res in ida_results:
        plt.plot(res['IM'], res['max_acc'], color='gray', linewidth=1, alpha=0.7)
        if res['collapsed']:
            collapse_idx = len(res['IM']) - 1
            plt.plot(res['IM'][collapse_idx], res['max_acc'][collapse_idx], 'ro')
    plt.xlabel('Intensity Measure (PGA, g)')
    plt.ylabel('Peak SDOF Acceleration (g)')
    plt.title('IDA Curves: Peak Acceleration vs. Intensity')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Plot Fragility Curve
    plt.figure(figsize=(6,5))
    plt.step(IM_points, prob_exceedance, where='post', label='Empirical Data', color='blue', linewidth=2, alpha=0.6)
    plt.plot(IM_plot, fragility_curve, label=f'Lognormal Fit (θ={median_theta:.2f}g, β={beta:.2f})', color='black', linewidth=2)
    plt.xlabel('Intensity Measure (PGA, g)')
    plt.ylabel('Probability of Collapse')
    plt.title('Fragility Curve for SDOF Collapse')
    plt.ylim([0,1.05])
    plt.xlim([0,2.0])
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()
